Versions

In [299]:
import platform

print('Python version: {}'.format(platform.python_version()))

Python version: 3.6.4


Import Libraries

In [176]:
from lxml import html
import requests
import pandas as pd

In [67]:
def findStars(x):
    x2 = str(x).replace('>', ' ').split()
    if ('bubble_5"' in x2):
        return 0.5
    elif ('bubble_10"' in x2):
        return 1
    elif ('bubble_15"' in x2):
        return 1.5
    elif ('bubble_20"' in x2):
        return 2
    elif ('bubble_25"' in x2):
        return 2.5
    elif ('bubble_30"' in x2):
        return 3
    elif ('bubble_35"' in x2):
        return 3.5
    elif ('bubble_40"' in x2):
        return 4
    elif ('bubble_45"' in x2):
        return 4.5
    elif ('bubble_50"' in x2):
        return 5
    else:
        return 0

In [7]:
page = requests.get('https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d2570383-Reviews-Cafe_in_the_Crypt-London_England.html')

In [9]:
top = html.fromstring(page.content)

In [16]:
print(html.tostring(top,pretty_print=True).decode('utf-8'))

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/" lang="en">
<head>
<meta http-equiv="content-type" content="text/html; charset=utf-8">
<link rel="stylesheet" type="text/css" href="https://static.tacdn.com/css2/long_lived_global_legacy-v22737519754a.css" data-rup="long_lived_global_legacy">
<link rel="icon" id="favicon" href="https://static.tacdn.com/favicon.ico" type="image/x-icon">
<link rel="preload" href="https://static.tacdn.com/css2/webfonts/TripAdvisor/TripAdvisor_Regular.woff2?v003.500" as="font" type="font/woff2" crossorigin>
<link rel="mask-icon" sizes="any" href="https://static.tacdn.com/img2/icons/ta_square.svg" color="#00a680">
<script type="text/javascript" data-rup="global_error">!function(){function e(e,t,n,o,i,a){var d={error_script:t,line:n,column:o,ready_state:document.readyState};return s?(require.defined("ta/util/Error")&&require("ta/util/Error").record(i,"error post load:: "+e,a,d,"ERROR",{isglobal:!0}),void r(e,i,"

In [42]:
top.xpath('//div[@title="ghfh"]/text()')

[]

In [160]:
# Get the reviews container containing all the reviews
reviews=top.find_class('innerBubble')
# Get the bubble which contains a review and all responses and comments to that review
#reviews = reviews[0].find_class('innerBubble')

In [161]:
# Loop through the review bubbles and only get the first instance of the 'entry' class.
# This means we do not consider the responses to the reviews, only the reviews themselves.
reviews_array = []
for i in reviews:
    reviews_array.append((i.find_class('entry')[0]).text_content())

In [162]:
reviews_array

['Stopped for lunch before going to the ballet at the Colesium so a perfect place to eat as only 2 minutes walk away. Lovely fresh made food, good choice of salads for my vegetarian daughter and delicious cakes. Really unique setting and although very busy...More',
 'My daughter had the fish and chips. The fish was ok but still had the skin on and the chips were soggy so for £12 not good. I had broccoli soup - pale green and only tasted of potatoes. My granddaughter is vegan and could...More',
 'Literally "dropped in" for an afternoon tea to round off an enjoyable afternoon with friends. Good selection of drinks and cakes as well as "cream teas". They were also serving an all-day Xmas lunch which looked very appetizing.',
 "Visited Cafe in the Crypt a week before Christmas. The food was OK - average. There are many people but the place seems to be well managed - tables kept clean etc, nice Christmas trees!. The 'dinner lady' who served me was cold and unapproachable,...More",
 'Our gro

In [163]:
ratings=top.find_class('review-container')

In [164]:
ratings_array = []
for i in ratings:
    ratings_array.append(findStars(html.tostring(i)))

In [165]:
ratings_array

[5, 3, 4, 3, 5, 5, 5, 5, 4, 2]

In [166]:
titles=top.find_class('noQuotes')

In [167]:
titles_array = []
for i in titles:
    titles_array.append(i.text_content())

In [192]:
titles_array

['Pre theatre lunch',
 'Late lunch disappointing',
 'Good for Afternoon Tea',
 'Good Average Place',
 'Christmas Lunch in the Crypt',
 'Lunch',
 'The Crypt can be relied on for good food at reasonable cost',
 'Unique ambience',
 'Nice little cafe! ',
 'Christmas treat']

In [169]:
dates=top.find_class('ratingDate')

In [170]:
dates_array=[]
for i in dates:
    dates_array.append(i.text_content())

In [171]:
dates_array

['Reviewed 3 days ago ',
 'Reviewed 4 days ago ',
 'Reviewed 1 week ago ',
 'Reviewed 1 week ago ',
 'Reviewed 1 week ago ',
 'Reviewed 2 weeks ago ',
 'Reviewed 2 weeks ago ',
 'Reviewed 2 weeks ago ',
 'Reviewed 2 weeks ago ',
 'Reviewed 3 weeks ago ']

In [212]:
# Diagnostics
def diagnostics(*args):
    '''
    This function checks that the lists given as arguments are of equal sizes
    args: An arbitrary number of lists
    '''
    [print(len(i)) for i in args]
    
    l = len(args[0])
    
    for i in args:
        if len(i) != l:
            print('Unequal Sizes!')
            return False
    return True
        

In [279]:
def webscrape(url,site):
    '''
    This functioni scrapes relevant review tags from a website url
    url: A string url
    site: A string indicating the site name to be scraped
    '''
    # Get the request object from the server
    page = requests.get(url)
    
    # Convert the request content to an html object
    top = html.fromstring(page.content)
    
    # These are the class names
    rev_class_1 = ''
    rev_class_2 = ''
    rat_class = ''
    title_class = ''
    dat_class=''
    
    if site.lower() == 'tripadvisor':
        rev_class_1 = 'innerBubble'
        rev_class_2 = 'entry'
        rat_class = 'review-container'
        titl_class = 'noQuotes'
        dat_class = 'ratingDate'
    else:
        print('The site {} is not supported'.format(site))
        return False
    
    # Get all the innerBubble classes which contain the reviews as well as any responses to these reviews
    reviews = top.find_class('innerBubble')
    
    # Loop through the review bubbles and only get the first instance of the 'entry' class
    # This means we do not consider the responses to the reviews, only the reviews themselves
    reviews_array = []
    for i in reviews:
        reviews_array.append((i.find_class('entry')[0]).text_content())
    
    # Get all the review containers
    ratings=top.find_class('review-container')
    
    # Within each review container is a class, the name of which determines the rating to display
    # We use the findStars function to determine the rating from the class name
    ratings_array = []
    for i in ratings:
        ratings_array.append(findStars(html.tostring(i)))
        
    # The titles of the reviews are within the 'noQuotes' tags
    titles=top.find_class('noQuotes')
    
    # Get the titles
    titles_array = []
    for i in titles:
        titles_array.append(i.text_content())
        
    # Get the rating date tags
    dates=top.find_class('ratingDate')
    
    # Get the dates
    dates_array=[]
    for i in dates:
        dates_array.append(i.text_content())
        
    # Diagnostics
    diagnostics(ratings_array,reviews_array,dates_array,titles_array)
    
    # Convert to a dataframe
    df_review = pd.DataFrame(reviews_array, columns=['Review'])
    df_ratings = pd.DataFrame(ratings_array, columns=['Rating'])
    df_titles = pd.DataFrame(titles_array, columns=['title'])
    df_reviewdates = pd.DataFrame(dates_array, columns=['date'])
    
    # Consolidate into a dataframe
    df_fullreview = pd.concat([df_review,df_titles,df_ratings['Rating'],df_reviewdates],axis=1)
    df_fullreview.dropna(inplace=True)
    
    # Combine review and title into a single column
    df_fullreview['fullreview'] = df_fullreview['Review'] + ' ' + df_fullreview['title']
    
    return df_fullreview

def fullscraper(site, first_url, url1, url2, increment_string1, increment_string2, total_pages, output_file):
    # Main data frame
    df = pd.DataFrame()
    
    # url incrementation differs per website
    if site == 'tripadvisor':
        df = webscrape(first_url,site)
        print('Getting reviews ' + str(1)+'/ '+str(total_pages))
        for i in range(1,total_pages):
            url_temp = url1 + increment_string1 + str(i) + increment_string2 + url2
            df_temp = webscrape(url_temp,site)
            df = pd.concat([df,df_temp])
            print('Getting reviews ' + str(i+1)+'/ '+str(total_pages))
    
    return df.reset_index().iloc[:,1:]

In [243]:
df = webscrape("https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d2570383-Reviews-Cafe_in_the_Crypt-London_England.html",'tripadvisor')

ConnectionError: HTTPSConnectionPool(host='www.tripadvisor.co.uk', port=443): Max retries exceeded with url: /Restaurant_Review-g186338-d2570383-Reviews-Cafe_in_the_Crypt-London_England.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002A79FA5D128>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [282]:
inurl1 = "https://www.tripadvisor.co.uk/Restaurant_Review-g186338-d2570383-Reviews"
inurl2 = "-Cafe_in_the_Crypt-London_England.html"

df_full = fullscraper(site='tripadvisor',first_url=inurl1+inurl2,url1=inurl1,url2=inurl2,increment_string1="-or",increment_string2="0",total_pages=2,output_file='testing.csv')

10
10
10
10
Getting reviews 1/ 2
10
10
10
10
Getting reviews 2/ 2


In [284]:
df_full

,Review,title,Rating,date,fullreview
0,Visited after walking through Trafalgar Square...,Quick drink and snack,5,Reviewed today,Visited after walking through Trafalgar Square...
1,Stopped for lunch before going to the ballet a...,Pre theatre lunch,5,Reviewed 3 days ago,Stopped for lunch before going to the ballet a...
2,My daughter had the fish and chips. The fish w...,Late lunch disappointing,3,Reviewed 4 days ago,My daughter had the fish and chips. The fish w...
3,"Literally ""dropped in"" for an afternoon tea to...",Good for Afternoon Tea,4,Reviewed 1 week ago,"Literally ""dropped in"" for an afternoon tea to..."
4,Visited Cafe in the Crypt a week before Christ...,Good Average Place,3,Reviewed 1 week ago,Visited Cafe in the Crypt a week before Christ...
5,Our group of 30 had Christmas lunch on 20/12/2...,Christmas Lunch in the Crypt,5,Reviewed 1 week ago,Our group of 30 had Christmas lunch on 20/12/2...
6,Affordable meals and pleasant surroundings. Th...,Lunch,5,Reviewed 2 weeks ago,Affordable meals and pleasant surroundings. Th...
7,Enjoyed delicious early Roast Turkey and veget...,The Crypt can be relied on for good food at re...,5,Reviewed 2 weeks ago,Enjoyed delicious early Roast Turkey and veget...
8,"I had no idea this place existed, but from the...",Unique ambience,5,Reviewed 2 weeks ago,"I had no idea this place existed, but from the..."
9,It’s always great to visit this cafe under Tra...,Nice little cafe!,4,Reviewed 2 weeks ago,It’s always great to visit this cafe under Tra...


In [34]:
inurl1 = "https://www.tripadvisor.co.uk/Attraction_Review-g186338-d188159-Reviews"
inurl2 = "-St_Paul_s_Cathedral-London_England.html"

df_full2 = fullscraper(site='tripadvisor',first_url=inurl1+inurl2,url1=inurl1,url2=inurl2,increment_string1="-or",increment_string2="0",total_pages=800,output_file='testing.csv')

Getting request 0/ 800
Getting request 1/ 800
Getting request 2/ 800
Getting request 3/ 800
Getting request 4/ 800
Getting request 5/ 800
Getting request 6/ 800
Getting request 7/ 800
Getting request 8/ 800
Getting request 9/ 800
Getting request 10/ 800
Getting request 11/ 800
Getting request 12/ 800
Getting request 13/ 800
Getting request 14/ 800
Getting request 15/ 800
Getting request 16/ 800
Getting request 17/ 800
Getting request 18/ 800
Getting request 19/ 800
Getting request 20/ 800
Getting request 21/ 800
Getting request 22/ 800
Getting request 23/ 800
Getting request 24/ 800
Getting request 25/ 800
Getting request 26/ 800
Getting request 27/ 800
Getting request 28/ 800
Getting request 29/ 800
Getting request 30/ 800
Getting request 31/ 800
Getting request 32/ 800
Getting request 33/ 800
Getting request 34/ 800
Getting request 35/ 800
Getting request 36/ 800
Getting request 37/ 800
Getting request 38/ 800
Getting request 39/ 800
Getting request 40/ 800
Getting request 41/ 800
Ge

Getting request 333/ 800
Getting request 334/ 800
Getting request 335/ 800
Getting request 336/ 800
Getting request 337/ 800
Getting request 338/ 800
Getting request 339/ 800
Getting request 340/ 800
Getting request 341/ 800
Getting request 342/ 800
Getting request 343/ 800
Getting request 344/ 800
Getting request 345/ 800
Getting request 346/ 800
Getting request 347/ 800
Getting request 348/ 800
Getting request 349/ 800
Getting request 350/ 800
Getting request 351/ 800
Getting request 352/ 800
Getting request 353/ 800
Getting request 354/ 800
Getting request 355/ 800
Getting request 356/ 800
Getting request 357/ 800
Getting request 358/ 800
Getting request 359/ 800
Getting request 360/ 800
Getting request 361/ 800
Getting request 362/ 800
Getting request 363/ 800
Getting request 364/ 800
Getting request 365/ 800
Getting request 366/ 800
Getting request 367/ 800
Getting request 368/ 800
Getting request 369/ 800
Getting request 370/ 800
Getting request 371/ 800
Getting request 372/ 800


Getting request 661/ 800
Getting request 662/ 800
Getting request 663/ 800
Getting request 664/ 800
Getting request 665/ 800
Getting request 666/ 800
Getting request 667/ 800
Getting request 668/ 800
Getting request 669/ 800
Getting request 670/ 800
Getting request 671/ 800
Getting request 672/ 800
Getting request 673/ 800
Getting request 674/ 800
Getting request 675/ 800
Getting request 676/ 800
Getting request 677/ 800
Getting request 678/ 800
Getting request 679/ 800
Getting request 680/ 800
Getting request 681/ 800
Getting request 682/ 800
Getting request 683/ 800
Getting request 684/ 800
Getting request 685/ 800
Getting request 686/ 800
Getting request 687/ 800
Getting request 688/ 800
Getting request 689/ 800
Getting request 690/ 800
Getting request 691/ 800
Getting request 692/ 800
Getting request 693/ 800
Getting request 694/ 800
Getting request 695/ 800
Getting request 696/ 800
Getting request 697/ 800
Getting request 698/ 800
Getting request 699/ 800
Getting request 700/ 800
